<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex03/patrick_ferreira/ex03_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
nome = 'Patrick de Carvalho Tavares Rezende Ferreira'
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


## Instruções

- Treinar uma rede neural de duas camadas como classificador binário na tarefa de análise de sentimentos usando dataset IMDB usando TF-IDF como entrada.

Deve-se implementar o laço de treinamento e validação da rede neural.

Neste exercício usaremos o IMDB com 20k exemplos para treino, 5k para desenvolvimento e 25k para teste.

# Importando os pacotes necessários

In [4]:
import collections
import os
import random
import re
import torch
import numpy as np

# Verificando se a GPU está disponível

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
   print(torch. cuda. get_device_name(dev))
else:
   dev = "cpu" 
print(dev)
device = torch.device(dev)

NVIDIA GeForce 940MX
cuda:0


## Preparando Dados

Primeiro, fazemos download do dataset:

In [6]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [7]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
y_valid = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
True This great film is composed mostly of documentary footage is currently contained on a DVD along with
False For all its visual delights, how much better Renaissance would have been in live action. The animati
False This movie in away was super-clever. It's theme rhymes with every single horror movie ever made. Val
3 últimas amostras treino:
False Michael Keaton has really never been a good actor; in the Tim Burton Batman movies he always falls i
False I have seen a lot of movies...this is the first one I ever walked out of the theater on. Don't even 
True I saw this film when I was a young child on television (thank-you Canadian Broadcasting Corporation)
3 primeiras amostras validação:
True Having read many of the other reviews for this film on the IMDb there is ostensibly a consensus amon
True It would help to know why it took so long for a book as movie-ready' as "The

### Definindo funções de manipulação de texto.

In [8]:
from typing import List

import re
import string


def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """

    # return re.sub('[' + string.punctuation + ']', '', text.lower()).split()

    pattern = r'\W+'

    return re.split(pattern, text.lower())

from collections import Counter


def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """

    tokens = []

    for t in texts:
        tokens.extend(tokenize(t))

    return dict(Counter(tokens).most_common(max_tokens))

def concatenate_list_of_str(texts: List[str]):
    return "".join(texts)

### Criando classe do dataset

In [13]:
from torch import nn
import torch

class TfIdfDataset(torch.utils.data.Dataset):
    def __init__(self, documents, max_tokens=1000):
        super().__init__()

        self.every_text = concatenate_list_of_str(documents)
        self.tokens_ocurrences = create_vocab(tokenize(self.every_text), max_tokens=max_tokens)

        self.lista_do_vocabulario = list(self.tokens_ocurrences.keys())

        self.tf = np.zeros((len(documents), len(self.lista_do_vocabulario)))
        for i, doc in enumerate(documents):
            tokenized_doc = tokenize(doc)[:-1]
            array_contagem_ocorrencias = np.array([0] * len(self.lista_do_vocabulario))
            for j, token in enumerate(self.lista_do_vocabulario):
                array_contagem_ocorrencias[j] += tokenized_doc.count(token)

            self.tf[i] = array_contagem_ocorrencias / len(tokenized_doc)

        self.idf_denominator = np.zeros((len(self.lista_do_vocabulario),))
        for i, doc in enumerate(documents):
            tokenized_doc = tokenize(doc)[:-1]
            for j, token in enumerate(self.lista_do_vocabulario):
                if token in tokenized_doc:
                    self.idf_denominator[j] += 1

        self.idf = np.log(len(documents) / self.idf_denominator)

### Testando dataset


In [14]:
x_assert = ["It is going to rain today.",
            "Today I am not going outside.",
            "I am going to watch the season premiere."]

y_assert = [False, True, True]

dataset = TfIdfDataset(x_assert, 8)